In [ ]:
import math
import numpy as np
import pandas as pd

import yfinance as yf

import datetime as dt
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# portfolio of tickers
tickers = ['TLT']
  
# start_date = "2015-01-01"
# end_date = dt.datetime.now().strftime("%Y-%m-%d")

# # Download historical data from yf API
# data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# # download data to csv
# data.to_csv(f"{tickers[0]}_data.csv")
     
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv(f"{tickers[0]}_data.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()
     

C:\Users\calli\AppData\Local\Temp\ipykernel_11568\1712973328.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
[*********************100%***********************]  1 of 1 completed


Ticker            TLT                                              
Price            Open        High        Low      Close      Volume
Date                                                               
2015-01-02  95.369545   96.404120  95.263822  96.147362   9432000.0
2015-01-05  96.668411   97.891783  96.630656  97.657684   9789500.0
2015-01-06  98.699788  100.051536  98.314659  99.417191  18331300.0
2015-01-07  98.699807   99.658867  98.299571  99.220871   9762900.0
2015-01-08  98.443039   98.450586  97.733184  97.906868   8055300.0